# 🩺 Medical Speech-to-Speech RAG Agent (Doctor–Patient, Multi-turn)

This notebook demonstrates a **low-latency speech-to-speech medical reasoning agent** with multi-turn memory and RAG, and runs a Gradio UI **inline**.

> ⚠️ Educational only, not a real medical device.

## 1. Install dependencies

In [ ]:
!pip install -r ../requirements.txt

## 2. Imports and setup

In [ ]:

import gradio as gr
from src.asr_tts import ASRWrapper, TTSWrapper
from src.rag_agent import MedicalRAGAgent

asr = ASRWrapper()
tts = TTSWrapper()
agent = MedicalRAGAgent()


## 3. Define chat functions (text and speech)

In [ ]:

from typing import List, Tuple

def chat_text(user_message: str, history: List[Tuple[str, str]]):
    if history is None:
        history = []
    if not user_message.strip():
        return "", history
    reply = agent.generate_response(user_message, history)
    history.append((user_message, reply))
    return "", history

def chat_speech(audio, history: List[Tuple[str, str]]):
    if history is None:
        history = []
    if audio is None:
        return "No audio detected.", None, history
    user_text = asr.speech_to_text(audio)
    reply = agent.generate_response(user_text, history)
    history.append((user_text, reply))
    sr, audio_out = tts.text_to_speech(reply)
    return reply, (sr, audio_out), history


## 4. Build Gradio UI (inline in notebook)

In [ ]:

with gr.Blocks() as demo:
    gr.Markdown("""
    # 🩺 Medical Speech-to-Speech RAG Agent (Demo - Notebook)

    - Speech ↔ Speech
    - RAG over tiny medical corpus
    - Multi-turn memory
    - Gradio inline UI
    """)

    with gr.Tab("Speech ↔ Speech"):
        state = gr.State([])
        audio_in = gr.Audio(source="microphone", type="numpy", label="Speak your symptoms")
        with gr.Row():
            text_out = gr.Textbox(label="Agent Response (Text)", lines=4)
            audio_out = gr.Audio(label="Agent Response (Audio)")

        audio_in.stop_recording(
            fn=chat_speech,
            inputs=[audio_in, state],
            outputs=[text_out, audio_out, state],
        )

    with gr.Tab("Text Chat"):
        state2 = gr.State([])
        chatbot = gr.Chatbot()
        msg = gr.Textbox(label="Describe your symptoms")

        def _chat_proxy(message, history):
            new_msg, new_hist = chat_text(message, history)
            return new_msg, new_hist

        msg.submit(_chat_proxy, [msg, chatbot], [msg, chatbot])

demo.launch(inline=True)
